# MusicGen fine tuning (with L4):

In [1]:
!apt-get update && apt-get install -y ffmpeg

!pip install av einops flashy hydra-core hydra_colorlog julius num2words sentencepiece spacy torchmetrics torchdiffeq xformers demucs omegaconf

%cd /content
!git clone https://github.com/facebookresearch/audiocraft

%cd /content/audiocraft
!git checkout v1.3.0

!pip install -e . --no-deps

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,205 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,287 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!cp "/content/drive/MyDrive/dataset_pena.zip" "/content/dataset.zip"
!unzip -q "/content/dataset.zip" -d "/content"

Mounted at /content/drive


In [14]:
import json
import glob
import os

OUTPUT_JSONL = "/content/audiocraft/dataset/data.jsonl"
DATASET_ZIP = "/content/drive/MyDrive/dataset_pena.zip"
DATASET_DIR = "/content/dataset_pena"

os.makedirs(os.path.dirname(OUTPUT_JSONL), exist_ok=True)

audio_files = glob.glob(f"{DATASET_DIR}/**/*.wav", recursive=True)
jsonl_data = []


for file_path in audio_files:
    try:

        txt_path = file_path.replace(".wav", ".txt")

        with open(txt_path, 'r', encoding='utf-8') as f:
            current_prompt = f.read().strip()

        entry = {
            "path": file_path,
            "duration": 30,
            "sample_rate": 32000,
            "amplitude": None,
            "weight": 1.0,
            "info_path": None,
            "description": current_prompt
        }
        jsonl_data.append(entry)

    except Exception as e:
        print(f"error: {e}")

with open(OUTPUT_JSONL, "w", encoding='utf-8') as f:
    for entry in jsonl_data:
        json.dump(entry, f)
        f.write('\n')

print(f"{OUTPUT_JSONL} file created with ({len(jsonl_data)} entries).")

/content/audiocraft/dataset/data.jsonl file created with (63 entries).


In [15]:
import os

with open("/content/audiocraft/dataset/data.jsonl", "r") as f:
    lines = f.readlines()

valid_lines = lines[:4]
train_lines = lines[4:]

if len(lines) < 5:
    valid_lines = lines[:1]
    train_lines = lines

with open("/content/audiocraft/dataset/train.jsonl", "w") as f:
    f.writelines(train_lines)

with open("/content/audiocraft/dataset/valid.jsonl", "w") as f:
    f.writelines(valid_lines)

In [16]:
config_content = """
# @package _global_

defaults:
  - musicgen/default
  - _self_

lm_model: transformer_lm
masking: span_masking

sample_rate: 32000
channels: 1
compression_model_checkpoint: //pretrained/facebook/encodec_32khz

transformer_lm:
  dim: 1024
  num_heads: 16
  num_layers: 24
  n_q: 4
  card: 2048
  activation: gelu
  norm_first: true

codebooks_pattern:
  modeling: delay
  delay:
    delays: [0, 0, 0, 0]
    flatten_first: 0

classifier_free_guidance:
  training_dropout: 0.1
  inference_scale: 3.0

attribute_dropout:
  args:
    active_on_eval: false
  text: {}

audio:
  sample_rate: 32000
  channels: 1

datasource:
  max_sample_rate: 32000
  max_channels: 1
  train: /content/audiocraft/dataset/train.jsonl
  valid: /content/audiocraft/dataset/valid.jsonl
  evaluate: /content/audiocraft/dataset/valid.jsonl
  generate: /content/audiocraft/dataset/valid.jsonl

optim:
  epochs: 30
  updates_per_epoch: 50
  lr: 1e-4

dataset:
  batch_size: 4
  num_workers: 0
  valid:
    num_workers: 0
    sample_on_weight: false
    sample_on_duration: false
    num_samples: 4
  evaluate:
    num_workers: 0
    sample_on_weight: false
    sample_on_duration: false
    num_samples: 4
  generate:
    num_workers: 0
    sample_on_weight: false
    sample_on_duration: false
    num_samples: 2

checkpoint:
  save_every: 10
  save_last: true
  save_best: true

logging:
  log_every: 10

schedule:
  lr_scheduler: cosine
  cosine:
    warmup: 4
    lr_min_ratio: 0.1
    cycle_length: 1.0

evaluate:
  every: 100
generate:
  every: 100
"""

import os
os.makedirs("/content/audiocraft/config/solver/custom", exist_ok=True)

with open("/content/audiocraft/config/solver/custom/mon_finetuning.yaml", "w") as f:
    f.write(config_content)


In [17]:
import os

target_file = "/content/audiocraft/audiocraft/modules/transformer.py"

with open(target_file, "r") as f:
    lines = f.readlines()

new_lines = []

for line in lines:
    if "assert self.causal or self.cross_attention," in line and "Streaming only available" in line:
        new_lines.append(f"# {line}")
    else:
        new_lines.append(line)

with open(target_file, "w") as f:
    f.writelines(new_lines)

In [18]:
import os
os.environ["USER"] = "colab_user"
os.makedirs("/content/checkpoints", exist_ok=True)

print("training in progress...")

!USER=colab_user python3 -m audiocraft.train \
    solver=custom/mon_finetuning \
    dora.dir=/content/checkpoints

training in progress...
2025-12-22 10:07:25.084054: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766398045.107182    8596 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766398045.116633    8596 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766398045.134281    8596 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766398045.134312    8596 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766398045.134315    8596 computation_placer.cc:177

In [19]:
import torch
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
from IPython.display import Audio, display
import glob
import os
import gc

gc.collect()
torch.cuda.empty_cache()

list_of_files = glob.glob(f'/tmp/audiocraft_colab_user/xps/4fbbbf33/checkpoint.th', recursive=True)

if not list_of_files:
    print("where is the checkpoint !")
else:
    fine_tuned_path = max(list_of_files, key=os.path.getctime)

    model = MusicGen.get_pretrained('facebook/musicgen-small')
    checkpoint = torch.load(fine_tuned_path, map_location='cpu', weights_only=False)

    if 'best_state' in checkpoint:
        state_dict = checkpoint['best_state']
    elif 'model' in checkpoint:
        state_dict = checkpoint['model']
    else:
        state_dict = checkpoint

    model.lm.load_state_dict(state_dict, strict=False)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


state_dict.bin:   0%|          | 0.00/841M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [21]:
prompts = ["Traditional Paso Doble music for bullfighting, festive banda, pena"]
model.set_generation_params(
    duration=30,
    temperature=1.0,
    top_k=250,
    cfg_coef=7
)

wav = model.generate(prompts)
display(Audio(wav[0].cpu().numpy(), rate=model.sample_rate))
